Obtain Data from API

In [97]:
import pandas as pd
import datetime 
import warnings
warnings.filterwarnings('ignore')

In [96]:
from bs4 import BeautifulSoup
import urllib2 
url = urllib2.urlopen("http://api.census.gov/data/timeseries/healthins/sahie/variables.html")

df_cols = pd.DataFrame()
soup = BeautifulSoup(url)
for a in soup.find_all('a', href=True):
    url = "http://api.census.gov/data/timeseries/healthins/sahie/" +  a['href']
    df_cols = df_cols.append(pd.DataFrame(pd.read_json(url,typ="series")).T)
    
df_cols = df_cols.reset_index(drop=True)
df_cols = df_cols[['name','label']]
df.cols = [x.lower() for x in df.columns]
df_cols

,name,label
0,AGE_DESC,Age Category Description
1,AGECAT,Age Category
2,COUNTY,County FIPS Code
3,for,Census API FIPS 'for' clause
4,GEOCAT,"Summary Level (040= state, 050= county)"
5,GEOID,State+County FIPS Code
6,in,Census API FIPS 'in' clause
7,IPR_DESC,Income Poverty Ratio Category Description
8,IPRCAT,Income Poverty Ratio Category
9,NAME,State or County Name


https://www.census.gov/did/www/sahie/data/files/SAHIE%20CSV%20File%20Layout.pdf

In [104]:
def get_sahie_data(geo = 'us', SEXCAT = '', start_date = 2010, end_date = 2013):
    
    
    cols = 'STABREV,NIC_PT,NUI_PT'
    url = build_url(cols,start_date, geo)
    df_temp = pd.read_json(url)
    df_temp.columns = df_temp.iloc[0]
    df = pd.DataFrame(columns = df_temp.columns)
    
    for dt in xrange(start_date, end_date + 1):
        url = build_url(cols, dt, geo)
        df_temp = pd.read_json(url)
        df_temp.columns = df_temp.iloc[0]
        df_temp = df_temp.iloc[1:]
        df = df.append(df_temp)

    df.time = pd.to_datetime(df.time) 
    df = df.set_index(df.time)
       
    if 'state' in df.columns: 
        df.pop('state')
    
    df.pop('time')
    df.index.name = 'date'
    
    df.columns = [x.lower() for x in df.columns]
    df = df.rename(columns = {'stabrev':'name'})
    
    return df

In [105]:
def build_url(cols,date, geo):
    date = str(date)
    api_key = '30b4db6d274abc643545d645254edaadb4e726dc'
    url = "http://api.census.gov/data/timeseries/healthins/sahie?get=%s&" \
    "for=%s:*&time=%s&key=%s" % (cols, geo, date, api_key)
    
    return url

In [106]:
df = get_sahie_data(geo='state')

In [107]:
df.head(3)

,name,nic_pt,nui_pt
date,,,
2010-01-01,AL,3339750,681437
2010-01-01,AK,508464,138777
2010-01-01,AZ,4357186,1042809


In [110]:
#API pulls al data as strings. Convert to floats so we can do maths.
df.nic_pt = pd.to_numeric(df.nic_pt) * 1.0
df.nui_pt = pd.to_numeric(df.nui_pt) * 1.0
df.dtypes

name       object
nic_pt    float64
nui_pt    float64
dtype: object

In [111]:
df['uninsured_pct'] = df.nui_pt / (df.nic_pt + df.nui_pt)
df.head(4)

,name,nic_pt,nui_pt,uninsured_pct
date,,,,
2010-01-01,AL,3339750,681437,0.169462
2010-01-01,AK,508464,138777,0.214413
2010-01-01,AZ,4357186,1042809,0.193113
2010-01-01,AR,1931198,500134,0.205704


In [112]:
names = df.name.unique()
max_year = df.index.max().year

df['uninsured_pct_change'] = 0.0
for x in names:
    df['uninsured_pct_change'][df.name==x] = df['uninsured_pct'][df.name==x].pct_change(periods=max_year-2010)

df[['name','uninsured_pct','uninsured_pct_change']].reset_index().sort(['name', 'date']).head(8)

,date,name,uninsured_pct,uninsured_pct_change
1,2010-01-01,AK,0.214413,NaN
52,2011-01-01,AK,0.217090,NaN
103,2012-01-01,AK,0.223264,NaN
154,2013-01-01,AK,0.208458,-0.027772
0,2010-01-01,AL,0.169462,NaN
51,2011-01-01,AL,0.166205,NaN
102,2012-01-01,AL,0.157667,NaN
153,2013-01-01,AL,0.158718,-0.063400


In [113]:
df[df.index.year == max_year].head(5)

,name,nic_pt,nui_pt,uninsured_pct,uninsured_pct_change
date,,,,,
2013-01-01,AL,3372259,636216,0.158718,-0.063400
2013-01-01,AK,520851,137170,0.208458,-0.027772
2013-01-01,AZ,4380044,1093575,0.199790,0.034576
2013-01-01,AR,1975350,458560,0.188405,-0.084097
2013-01-01,CA,26596092,6382717,0.193540,-0.063896


If you just want the entire CSV

In [130]:
import os
import csv
import pandas as pd

start_date = 2008
end_date = 2013

filename = 'sahie.csv'
try:
    os.remove(filename)
except OSError:
    pass

for yr in xrange(start_date, end_date + 1):
    
    #In 2011, the web link contains an underscore. 
    if yr == 2011:
        url = "https://www.census.gov/did/www/sahie/data/files/sahie_%s.csv" % str(yr)
    else:
        url = "https://www.census.gov/did/www/sahie/data/files/sahie%s.csv" % str(yr)
    
    df = pd.read_csv(url,skiprows=3,header=0) 
    
    #Column headers have spaces, let's remove them
    df = df.rename(columns=lambda x: x.strip())
    
    if yr == start_date:
        df.to_csv('sahie.csv',index=False, header=True)
    else:
        with open('sahie.csv', 'a') as f:
            df.to_csv(f, index = False, header=False)
            
    print yr

2008
2009
2010
2011
2012
2013


In [19]:
dfc.head()

,year,stcou,geocat,agecat,racecat,sexcat,iprcat,name,nipr,nipr_moe,nui,nui_moe,nic,nic_moe,pctui,pctui_moe,pctic,pctic_moe
0,2013,"=""01000""",40,0,0,0,0,Alabama ...,4008475,NaN,636216,14701,3372259,14701,15.9,0.4,84.1,0.4
1,2013,"=""02000""",40,0,0,0,0,Alaska ...,658020,NaN,137170,5322,520851,5322,20.8,0.8,79.2,0.8
2,2013,"=""04000""",40,0,0,0,0,Arizona ...,5473619,NaN,1093575,21161,4380044,21161,20.0,0.4,80.0,0.4
3,2013,"=""05000""",40,0,0,0,0,Arkansas ...,2433909,NaN,458560,11936,1975350,11936,18.8,0.5,81.2,0.5
4,2013,"=""06000""",40,0,0,0,0,California ...,32978809,NaN,6382717,67813,26596092,67813,19.4,0.2,80.6,0.2


The definitions to our columns are located at the following link. Let's put them into a table.
http://api.census.gov/data/timeseries/healthins/sahie/variables.html